<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Tutorial/2-Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine learning with coral reef soundscape data**

This notebook is a supporting tutorial for the study **Unlocking the soundscape of coral reefs with artificial intelligence** by [Williams et al (2024a)](https://www.biorxiv.org/content/10.1101/2024.02.02.578582v1). If you use any of these methods after reading this then please cite the article.

In this publication we recommend combining pretrained neural networks with unsupervised learning for analysing soundscape ecology.

## What this notebook does:
1. Set up: access some sample data and install the required packages we need.
2. Extract features from the audio data using the SurfPerch pretrained neural network.


## **SurfPerch: A pretrained neural network fine tuned to coral reefs**

In the associated study to this tutorial we used VGGish. However, here we will use SurfPerch, a newly developed pretrained neural network fine tuned to coral reefs which we created in a collaboration with Google DeepMind. It was created and rigorously tested on audio data from 16 unique datasets across 12 countries. You can read more about the network in its supporting research article **Leveraging tropical reef, bird and unrelated sounds for superior transfer learning in marine bioacoustics**, [Williams et al (2024b)](https://arxiv.org/abs/2404.16436).

SurfPerch can also be used to rapdily identify individual sounds in your data, as opposed to the whole soundscape approach presented in Unlocking the soundscape of coral reefs with artificial intelligence by [Williams et al (2024a)](https://www.biorxiv.org/content/10.1101/2024.02.02.578582v1). See a full tutorial on identifying individual sounds [here](https://github.com/BenUCL/surfperch/blob/surfperch/SurfPerch_Demo_with_Calling_in_Our_Corals.ipynb).


## **Our sample data**
We'll use a small sample dataset for this tutorial. This data consists of 262 audio files from healthy, degraded and restored coral reefs in Indonesia. These reefs are part of the worlds largest coral reef restoration program [buildingcoral.com](https://www.buildingcoral.com/). See [Williams et al (2022) ](https://doi.org/10.1016/j.ecolind.2022.108986) for more detail on this audio.







# **Step 1: Set up**

## **Access sample data**

There are two routes to accessing the sample data.

### **Route 1 (the quick and easy route)**
The sample data is held in this [public GDrive folder](https://drive.google.com/drive/folders/1JDqpHaUyVxFNuw3K_y9J7f9g28oBk3v5?usp=sharing). To access it:
1. Click this link.
2. Click on the dropdown arrow next to the 'sample_data' heading.
3. Click organize -> Add shortcut.
4. Select the 'All locations' tab.
5. Select 'MyDrive' -> 'Add'

This will add a link to the folder in your GDrive without taking up any of your own GDrive space.


### **Route 2**
If for any reason route 1 no longer works, this sample dataset is held within the `audio_dir` folder within the `Reef-acoustics-and-AI` zip file on the [Zenodo repository](https://doi.org/10.5281/zenodo.11106482) for the this study (make sure you have accessed the most up to date version of this repo from the tab on the right):

1. Download the tutorial file.
2. Unzip the file.
3. Upload all the audio files from the sample data folder into a folder called 'sample_data' on your GDrive. Note, make sure this is in the MyDrive folder on your GDrive, and not in a sub folder.


## **Access the pretrained model**
### **Route 1**
As above, the model can be accessed from a public [GDrive folder](https://drive.google.com/drive/folders/1PzxO1dcjMtIVdqBqEDBBlUQHf-P22EkD?usp=sharing).

Once again, create a shortcut to this in your MyDrive folder.


### **Route 2**
The model can also be accessed from a [Zenodo repository](https://doi.org/10.5281/zenodo.11071202).

1. Download the SurfPerch.zip file.
2. Unzip this file.
3. Find the 'Saved model' folder (note not saved_model.pb) inside this.
4. Rename the 'Saved model' folder 'SurfPerch-model'.
3. Upload this 'SurfPerch-model'folder to your MyDrive folder in Google Drive.


In [22]:
#@title Import packages
import os # for handling files and directories
import librosa # for audio processing
import tensorflow as tf # for machine learning
import tensorflow_hub as hub # for machine learning
import numpy as np # for numerical processing
import pandas as pd # for handling dataframes
from tqdm import tqdm # for progress bar

In [23]:
#@title Set all filepaths

# Directory containing audio files (look for the shortcut link you added in GDrive)
sample_audio = "/mnt/d/Aqoustics/UMAP/test"

# Path to pretained network (look for the shortcut link you added in GDrive)
model_path = '/home/os/aqoustics/Aqoustics-Surfperch/kaggle'

# Path where a csv of extracted features will be saved
results_path = '/home/os/aqoustics/Aqoustics-Surfperch/data/output_dir/'
if not os.path.exists(results_path):
  os.mkdir(results_path)

### Load the SurfPerch neural network model
 Check we have the savedmodel folder in GDrive, you should see:

 assets	saved_model.pb	variables

In [24]:
# Check model is present
!ls '/home/os/aqoustics/Aqoustics-Surfperch/kaggle'

assets	saved_model.pb	variables


In [25]:
# We will load the pretrained neural net as 'model'
model = tf.saved_model.load(model_path)
model.signatures


_SignatureMap({'serving_default': <ConcreteFunction (*, inputs: TensorSpec(shape=(None, 160000), dtype=tf.float32, name='inputs')) -> Dict[['fsd50k_label', TensorSpec(shape=(None, 200), dtype=tf.float32, name='fsd50k_label')], ['embedding', TensorSpec(shape=(None, 1280), dtype=tf.float32, name='embedding')], ['frontend', TensorSpec(shape=(None, 500, 128), dtype=tf.float32, name='frontend')], ['reef_label', TensorSpec(shape=(None, 38), dtype=tf.float32, name='reef_label')], ['family', TensorSpec(shape=(None, 249), dtype=tf.float32, name='family')], ['genus', TensorSpec(shape=(None, 2333), dtype=tf.float32, name='genus')], ['order', TensorSpec(shape=(None, 41), dtype=tf.float32, name='order')], ['label', TensorSpec(shape=(None, 10932), dtype=tf.float32, name='label')]] at 0x7F9A282B5540>})

# **Step 2: Extract features with the neural net**

Now we run the main for loop to iterate over each file extract features using the pretrained nereul network.

The results will be saved to a 'pandas dataframe', similar to a dataframe in R, and, to the 'extracted_features.csv' which should appear in the file tab on the left.

In [26]:
#@title Define helper functions for inference
def get_sample_rate(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)
    return sample_rate


def resample_and_split_audio(file_path, original_sr, target_sr=32000, segment_duration=5):
    audio, _ = librosa.load(file_path, sr=original_sr)  # Load with original sample rate
    audio = librosa.resample(audio, orig_sr=original_sr, target_sr=target_sr)  # Resample to 32kHz
    segments = []

    segment_length = target_sr * segment_duration
    total_segments = len(audio) // segment_length

    for i in range(total_segments):
        start = i * segment_length
        end = start + segment_length
        segments.append(audio[start:end])

    return segments


def process_audio_files(audio_dir, model):
    rows_list = []
    original_sr = None

    # Loop through every file in audio_dir
    for filename in tqdm(os.listdir(audio_dir), desc="Processing audio files"):
        if filename.lower().endswith('.wav'):
            file_path = os.path.join(audio_dir, filename)

            # Check if the sample rate has not been set yet
            if original_sr is None:
                original_sr = get_sample_rate(file_path)  # Get the sample rate from the first file

            try:
                segments = resample_and_split_audio(file_path, original_sr=original_sr)
                
                for i, segment in enumerate(segments):
                    # Model expects batch dimension, so use np.newaxis to add it
                    output = model.infer_tf(segment[np.newaxis, :])
                    
                    # Extract the embedding tensor
                    embeddings = output['embedding']
                    
                    # Convert to numpy array
                    embedding = embeddings.numpy()[0]
                    
                    embedding_index = i + 1
                    row_data = {'filename': filename, 'embedding_index': embedding_index}
                    for j, feature in enumerate(embedding):
                        row_data[f'feature_{j}'] = feature
                    rows_list.append(row_data)
            except Exception as e:
                print(f"An error occurred while processing file: {filename}. Error: {e}")

    features_df = pd.DataFrame(rows_list)
    return features_df


## Run feature extraction and save results to a csv

This will run orders of magnitudes faster if using GPU instance of Google colab. Check your runtime type if unsure.

In [27]:
# Extract the features
features_df = process_audio_files(sample_audio, model)

# Save results to GDrive
#features_df_path = results_path + 'surfperch_feature_embeddings.csv'
#features_df.to_csv(features_df_path, index=False)

Processing audio files: 100%|██████████| 8314/8314 [01:05<00:00, 126.16it/s]


You should now have the folder `MyDrive/AI4Reefs-tutorial-results/` in your GDrive. In here there should be a `surfperch_feature_embeddings.csv` file.


Now we're done with the GPU on colab, its good practice to switch back to a standard CPU runtime. You'll need to rerun the imports, reconnect your GDrive and run the filepaths cell again if doing so.

In [28]:
#@title Take a peek at the features dataframe

# Load the saved csv from gdrive as a dataframe
#features_df = pd.read_csv(results_path + 'test1.csv')

features_df

,filename,embedding_index,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_1270,feature_1271,feature_1272,feature_1273,feature_1274,feature_1275,feature_1276,feature_1277,feature_1278,feature_1279
0,clip_ind_D1_20220829_120000_0.wav,1,0.015825,-0.014295,-0.041377,0.059468,-0.098099,0.136571,0.226944,-0.026446,...,-0.012807,-0.057299,0.137240,-0.063180,-0.019601,0.001133,0.041733,-0.014994,0.044510,0.022710
1,clip_ind_D1_20220829_120000_1.wav,1,0.091357,-0.000085,-0.048566,-0.002486,-0.159877,0.120174,0.189179,-0.050407,...,-0.001316,0.034789,0.114856,-0.055004,-0.020059,0.017637,0.044497,-0.029143,0.047238,0.032173
2,clip_ind_D1_20220829_120000_2.wav,1,0.026794,-0.000204,-0.056221,0.122827,-0.134732,0.129436,0.245470,-0.050055,...,-0.012710,-0.064497,0.072393,-0.028102,-0.022759,0.026983,-0.100695,-0.032024,0.048788,0.018631
3,clip_ind_D1_20220829_120000_3.wav,1,0.092736,-0.009935,-0.032423,0.002704,-0.151599,0.062451,0.255388,-0.054694,...,-0.020238,-0.079634,0.064258,-0.112389,-0.017458,0.024369,-0.080522,-0.028680,0.061943,0.032965
4,clip_ind_D1_20220829_120000_4.wav,1,0.066163,0.057435,-0.074894,0.022301,-0.120259,0.112542,0.230405,-0.028742,...,0.002856,-0.049592,0.056021,-0.063968,-0.018785,0.019826,-0.001366,-0.023779,0.029028,0.036399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8307,clip_ind_R6_20220922_192000_5.wav,1,-0.006828,-0.057824,-0.078678,0.055846,-0.136344,0.163978,0.028686,-0.034856,...,-0.011053,-0.018507,0.096655,-0.110390,-0.017311,-0.009239,-0.013894,0.011521,-0.035399,0.042077
8308,clip_ind_R6_20220922_192000_6.wav,1,0.003340,-0.031082,-0.044733,0.027860,-0.093893,0.134824,0.152387,-0.036256,...,-0.013895,-0.036536,0.117107,-0.116105,-0.024471,-0.016544,0.021120,-0.004177,0.002125,0.070574
8309,clip_ind_R6_20220922_192000_7.wav,1,0.006566,-0.061826,0.035667,0.012489,-0.169057,0.126389,0.097763,0.000254,...,0.018961,-0.064228,0.235332,0.011827,-0.014588,0.016123,-0.029384,-0.025540,0.054214,0.025265
8310,clip_ind_R6_20220922_192000_8.wav,1,0.129206,0.013581,-0.038946,-0.037526,-0.157498,0.069277,0.116801,-0.017786,...,-0.000074,-0.066095,0.253978,0.078425,-0.009302,0.011873,-0.033377,-0.031343,0.104150,0.019069


## **Finished!**

You should see a results table that contains:
1. All the audio files in our sample data under the 'filename' column.
2. SurfPerch cuts audio files into 5s chunks, the chunk which each rows corresponds to is under 'embedding_index'.
3. There should be feature columns running from feature_0 to feature_1279. Each 5s chunk is now represented by these highly informative (to a machine) feature embeddings.
